#SocialAL Model
#generate trial-by-trial output from model fits

KLS 2.7.20
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

Python version and modules

In [58]:
import sys
print(sys.version)  
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


Define Functions

In [59]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    tletters = ["A", "B", "C"]
    
    totalLLH = 0 
    for trial in range(0, len(data)):

        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
           
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
            

            # update prob and value for trustee 
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)         
            ev[trustee] = update_value(prob[trustee])
            
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(trustee)
        for x in options:
            prob[x] = prob[x] + decay * (0.5 - prob[x])
            ev[x] =  update_value(prob[x])
        
        print("Trial #: ", trial + 1)
        #print("Trustee: ", tletters[trustee])
        #print('EV for Trustee:', ev[trustee])
        #print('EV for all:', ev)
        #print('choice prob: ', cprob)
        #print('Probs after decay: ', prob) 
        print('LLH: ', totalLLH)
        #print('Trustee Prob: ', prob[trustee])
        #print('Trustee Expected values: ', ev[trustee])
        
        
    return totalLLH

def model_fit_once(data, param):
        # initialize free parameters with randomly chosen numbers
        a_gain=param[0]
        a_loss=param[1]
        beta=param[2]
        decay = param[3]
        params = [a_gain, a_loss, beta, decay]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 100), (0, 1)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

Get data

In [60]:
#dt = pd.read_csv('../data/modeling/sub-1004.csv')
#dt = pd.read_csv('../data/modeling/sub-1008.csv') #- try getting this sub to work
dt = pd.read_csv('../data/modeling/sub-2032.csv') #- try getting this sub to work
# recode trial type into numbers for model
def stims(trial_type):
    if trial_type == "Trustworthy":
        return 0
    elif trial_type == "Neutral":
        return 1
    elif trial_type == "Untrustworthy":
        return 2
dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
# rename response_key to choice
def choices(response_key):
    if response_key == 'None':
        return 0 
    else:
        return response_key  
dt['Choice'] = dt['response_key'].apply(choices)
dt['Choice'] = pd.to_numeric(dt['Choice'])
# calculte the trustee response
def resp(trial_earnings):
    if trial_earnings >= 12:
        return 1
    else:
        return 0
dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
#print(data)

Input parameters

In [61]:
#array = [0.32694469, 0.36339119, 0.61381453, 0.03077456, 42.16444278393116] # 1004
#array = [ 0.09417835,  0.98770281,  0.53653418,  0.56243127, 41.59311547] # 1008
array = [4.05005864e-02, 0.00000000e+00, 1.87113895e+00, 1.62348280e-01, 5.41170868e+01] #2032

In [62]:
model_fit_once(data, array)

Trial #:  1
LLH:  0
Trial #:  2
LLH:  1.3862943611198906
Trial #:  3
LLH:  1.3862943611198906
Trial #:  4
LLH:  2.772588722239781
Trial #:  5
LLH:  4.1588830833596715
Trial #:  6
LLH:  5.545177444479562
Trial #:  7
LLH:  7.027079096331515
Trial #:  8
LLH:  7.027079096331515
Trial #:  9
LLH:  8.491315856543082
Trial #:  10
LLH:  9.877610217662973
Trial #:  11
LLH:  11.40672063815612
Trial #:  12
LLH:  12.712248454626799
Trial #:  13
LLH:  14.09854281574669
Trial #:  14
LLH:  15.413009175715125
Trial #:  15
LLH:  16.799303536835016
Trial #:  16
LLH:  18.010767673319247
Trial #:  17
LLH:  18.010767673319247
Trial #:  18
LLH:  19.073688468915783
Trial #:  19
LLH:  20.859667034624362
Trial #:  20
LLH:  22.245961395744253
Trial #:  21
LLH:  23.21469903939777
Trial #:  22
LLH:  24.600993400517662
Trial #:  23
LLH:  26.055781172820783
Trial #:  24
LLH:  26.965405352709265
Trial #:  25
LLH:  26.965405352709265
Trial #:  26
LLH:  28.027647927089664
Trial #:  27
LLH:  29.312321999454426
Trial #: 

Trial #:  8
LLH:  7.044099953398925
Trial #:  9
LLH:  8.522040117950889
Trial #:  10
LLH:  9.90833447907078
Trial #:  11
LLH:  11.464010753322775
Trial #:  12
LLH:  12.762710119306048
Trial #:  13
LLH:  14.149004480425939
Trial #:  14
LLH:  15.456381575957987
Trial #:  15
LLH:  16.84267593707788
Trial #:  16
LLH:  18.02884855320113
Trial #:  17
LLH:  18.02884855320113
Trial #:  18
LLH:  19.049368661764056
Trial #:  19
LLH:  20.90134055584234
Trial #:  20
LLH:  22.28763491696223
Trial #:  21
LLH:  23.2051383699096
Trial #:  22
LLH:  24.591432731029492
Trial #:  23
LLH:  26.058440644330364
Trial #:  24
LLH:  26.912278660346363
Trial #:  25
LLH:  26.912278660346363
Trial #:  26
LLH:  27.931906874975017
Trial #:  27
LLH:  29.2215412637727
Trial #:  28
LLH:  30.60783562489259
Trial #:  29
LLH:  31.56408435912696
Trial #:  30
LLH:  32.950378720246846
Trial #:  31
LLH:  34.33667308136673
Trial #:  32
LLH:  35.245550616091464
Trial #:  33
LLH:  36.63184497721135
Trial #:  34
LLH:  38.018139338

LLH:  26.052826510837257
Trial #:  24
LLH:  26.948556392920864
Trial #:  25
LLH:  26.948556392920864
Trial #:  26
LLH:  28.000235771992738
Trial #:  27
LLH:  29.28554180805871
Trial #:  28
LLH:  30.6718361691786
Trial #:  29
LLH:  31.662066141578293
Trial #:  30
LLH:  33.04836050269818
Trial #:  31
LLH:  34.43465486381807
Trial #:  32
LLH:  35.381379765493435
Trial #:  33
LLH:  36.76767412661332
Trial #:  34
LLH:  38.15396848773321
Trial #:  35
LLH:  39.61326436728572
Trial #:  36
LLH:  40.54619144530914
Trial #:  37
LLH:  41.93248580642903
Trial #:  38
LLH:  43.23575280707964
Trial #:  39
LLH:  44.62204716819953
Trial #:  40
LLH:  45.906657529487795
Trial #:  41
LLH:  47.200711897953994
Trial #:  42
LLH:  49.49701984807369
Trial #:  43
LLH:  51.27735074997999
Trial #:  44
LLH:  52.91942176755593
Trial #:  45
LLH:  54.30571612867582
Trial #:  1
LLH:  0
Trial #:  2
LLH:  1.3862943611198906
Trial #:  3
LLH:  1.3862943611198906
Trial #:  4
LLH:  2.772588722239781
Trial #:  5
LLH:  4.15888

Trial #:  29
LLH:  31.634949107563973
Trial #:  30
LLH:  33.02124346868386
Trial #:  31
LLH:  34.40753782980375
Trial #:  32
LLH:  35.349346441955305
Trial #:  33
LLH:  36.73564080307519
Trial #:  34
LLH:  38.12193516419508
Trial #:  35
LLH:  39.59234761106443
Trial #:  36
LLH:  40.52976111357907
Trial #:  37
LLH:  41.91605547469896
Trial #:  38
LLH:  43.21632194591219
Trial #:  39
LLH:  44.60261630703208
Trial #:  40
LLH:  45.887147838517265
Trial #:  41
LLH:  47.179968204289516
Trial #:  42
LLH:  49.46425750279033
Trial #:  43
LLH:  51.26517597540891
Trial #:  44
LLH:  52.91529668507457
Trial #:  45
LLH:  54.30159104619446
Trial #:  1
LLH:  0
Trial #:  2
LLH:  1.3862943611198906
Trial #:  3
LLH:  1.3862943611198906
Trial #:  4
LLH:  2.772588722239781
Trial #:  5
LLH:  4.1588830833596715
Trial #:  6
LLH:  5.545177444479562
Trial #:  7
LLH:  7.0238473306833
Trial #:  8
LLH:  7.0238473306833
Trial #:  9
LLH:  8.487293917371344
Trial #:  10
LLH:  9.873588278491235
Trial #:  11
LLH:  11.4

Trial #:  10
LLH:  9.873457895087197
Trial #:  11
LLH:  11.404069871096775
Trial #:  12
LLH:  12.709166122662646
Trial #:  13
LLH:  14.095460483782537
Trial #:  14
LLH:  15.408447727636254
Trial #:  15
LLH:  16.794742088756145
Trial #:  16
LLH:  17.993050469460538
Trial #:  17
LLH:  17.993050469460538
Trial #:  18
LLH:  19.04490818434256
Trial #:  19
LLH:  20.841521685229797
Trial #:  20
LLH:  22.227816046349687
Trial #:  21
LLH:  23.1865492544619
Trial #:  22
LLH:  24.572843615581792
Trial #:  23
LLH:  26.03649398893156
Trial #:  24
LLH:  26.93421080705339
Trial #:  25
LLH:  26.93421080705339
Trial #:  26
LLH:  27.986388307788168
Trial #:  27
LLH:  29.270804728707315
Trial #:  28
LLH:  30.657099089827206
Trial #:  29
LLH:  31.63505622966169
Trial #:  30
LLH:  33.02135059078158
Trial #:  31
LLH:  34.40764495190147
Trial #:  32
LLH:  35.349523139877036
Trial #:  33
LLH:  36.73581750099692
Trial #:  34
LLH:  38.12211186211681
Trial #:  35
LLH:  39.59258395227346
Trial #:  36
LLH:  40.530

...

Trial #:  4
LLH:  2.772588722239781
Trial #:  5
LLH:  4.1588830833596715
Trial #:  6
LLH:  5.545177444479562
Trial #:  7
LLH:  7.036933478633424
Trial #:  8
LLH:  7.036933478633424
Trial #:  9
LLH:  8.516875696921218
Trial #:  10
LLH:  9.903170058041109
Trial #:  11
LLH:  11.45066085934329
Trial #:  12
LLH:  12.751288142900588
Trial #:  13
LLH:  14.137582504020479
Trial #:  14
LLH:  15.443239285775858
Trial #:  15
LLH:  16.829533646895747
Trial #:  16
LLH:  17.97165600586366
Trial #:  17
LLH:  17.97165600586366
Trial #:  18
LLH:  18.995001411236657
Trial #:  19
LLH:  20.839607178385936
Trial #:  20
LLH:  22.225901539505827
Trial #:  21
LLH:  23.183895854405726
Trial #:  22
LLH:  24.570190215525617
Trial #:  23
LLH:  26.068669179908103
Trial #:  24
LLH:  26.990794537161776
Trial #:  25
LLH:  26.990794537161776
Trial #:  26
LLH:  28.007134236475867
Trial #:  27
LLH:  29.291102157519322
Trial #:  28
LLH:  30.677396518639213
Trial #:  29
LLH:  31.655450398631437
Trial #:  30
LLH:  33.04174

Trial #:  23
LLH:  26.07627479445956
Trial #:  24
LLH:  26.998270239224837
Trial #:  25
LLH:  26.998270239224837
Trial #:  26
LLH:  28.0023632599357
Trial #:  27
LLH:  29.286410674309003
Trial #:  28
LLH:  30.672705035428894
Trial #:  29
LLH:  31.643609381463442
Trial #:  30
LLH:  33.02990374258333
Trial #:  31
LLH:  34.416198103703216
Trial #:  32
LLH:  35.36673147419187
Trial #:  33
LLH:  36.75302583531176
Trial #:  34
LLH:  38.13932019643165
Trial #:  35
LLH:  39.65465537902152
Trial #:  36
LLH:  40.592868101533895
Trial #:  37
LLH:  41.97916246265378
Trial #:  38
LLH:  43.27382141728725
Trial #:  39
LLH:  44.66011577840714
Trial #:  40
LLH:  45.94568750950212
Trial #:  41
LLH:  47.236049218603966
Trial #:  42
LLH:  49.34993526275479
Trial #:  43
LLH:  51.214784560316225
Trial #:  44
LLH:  52.849743323137645
Trial #:  45
LLH:  54.23603768425753
Trial #:  1
LLH:  0
Trial #:  2
LLH:  1.3862943611198906
Trial #:  3
LLH:  1.3862943611198906
Trial #:  4
LLH:  2.772588722239781
Trial #:  

Trial #:  10
LLH:  9.916291199020382
Trial #:  11
LLH:  11.471879891759372
Trial #:  12
LLH:  12.770599099293225
Trial #:  13
LLH:  14.156893460413116
Trial #:  14
LLH:  15.45979037350715
Trial #:  15
LLH:  16.84608473462704
Trial #:  16
LLH:  17.966399925201365
Trial #:  17
LLH:  17.966399925201365
Trial #:  18
LLH:  18.977067412153545
Trial #:  19
LLH:  20.84256535075478
Trial #:  20
LLH:  22.22885971187467
Trial #:  21
LLH:  23.18189731178608
Trial #:  22
LLH:  24.56819167290597
Trial #:  23
LLH:  26.081299890588802
Trial #:  24
LLH:  27.004489381360024
Trial #:  25
LLH:  27.004489381360024
Trial #:  26
LLH:  28.00640788590074
Trial #:  27
LLH:  29.290472639617406
Trial #:  28
LLH:  30.676767000737296
Trial #:  29
LLH:  31.648622400197002
Trial #:  30
LLH:  33.03491676131689
Trial #:  31
LLH:  34.42121112243678
Trial #:  32
LLH:  35.37298669476731
Trial #:  33
LLH:  36.7592810558872
Trial #:  34
LLH:  38.14557541700709
Trial #:  35
LLH:  39.66176507821221
Trial #:  36
LLH:  40.60001

Trial #:  38
LLH:  43.282577594143085
Trial #:  39
LLH:  44.66887193652346
Trial #:  40
LLH:  45.95454126328205
Trial #:  41
LLH:  47.244995706107986
Trial #:  42
LLH:  49.3523417049905
Trial #:  43
LLH:  51.216426629361635
Trial #:  44
LLH:  52.849493731096416
Trial #:  45
LLH:  54.23578804990542
Trial #:  1
LLH:  0
Trial #:  2
LLH:  1.3862943611198906
Trial #:  3
LLH:  1.3862943611198906
Trial #:  4
LLH:  2.772588722239781
Trial #:  5
LLH:  4.1588830833596715
Trial #:  6
LLH:  5.545177444479562
Trial #:  7
LLH:  7.043150071021186
Trial #:  8
LLH:  7.043150071021186
Trial #:  9
LLH:  8.5302964639217
Trial #:  10
LLH:  9.91659082504159
Trial #:  11
LLH:  11.472217042581043
Trial #:  12
LLH:  12.770927746839305
Trial #:  13
LLH:  14.157222107959196
Trial #:  14
LLH:  15.460141218356567
Trial #:  15
LLH:  16.846435579476456
Trial #:  16
LLH:  17.967205994454293
Trial #:  17
LLH:  17.967205994454293
Trial #:  18
LLH:  18.978115203725295
Trial #:  19
LLH:  20.843379560875608
Trial #:  20
L

      fun: 54.23578799756266
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.89226454e-05, 5.23427630e+00, 2.13162821e-06, 2.27373675e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 270
      nit: 40
   status: 0
  success: True
        x: array([0.35955093, 0.        , 0.20076404, 0.08299769])